In [2]:
from sklearn.datasets import load_iris
X,y = load_iris(return_X_y=True)
X.shape,y.shape

((150, 4), (150,))

# **GMM IMPLEMENTATION**

### **E-STEP**

In [0]:
#Computing Gamma Values
from scipy.stats import multivariate_normal as mvn

def getGammaValues(X,num_of_clusters,mixing_coefficients,mean_vectors,covariance_matrices):
    gamma_values = []
    for i in range(num_of_clusters):
      gamma_values.append(mixing_coefficients[i] * mvn.pdf(X, mean_vectors[i],covariance_matrices[i]))
    gamma_norm = np.sum(gamma_values, axis=0)[:,np.newaxis]
    gamma_values /= gamma_norm.T
    return gamma_values

### **M-STEP**

In [0]:
#Computing mixing_coefficient,mean_vectors and covariance_matrices
def computeMixCoeffMeanVecAndCov(X,gamma_values,num_of_clusters):
    mixing_coefficients = np.mean(gamma_values, axis = 1)
    print("Mixing Coeffiients: \n",mixing_coefficients)
    mean_vectors = np.dot(gamma_values, X) 
    mean_vectors /= np.sum(gamma_values.T, axis = 0)[:,np.newaxis]
    print("Mean Vectors: \n",mean_vectors)
    covariance_matrices = []
    for i in range(num_of_clusters):
            x = X - mean_vectors[i] # (N x d)
            gamma_diag = np.diag(gamma_values[i])
            x_mu = np.matrix(x)
            gamma_diag = np.matrix(gamma_diag)
            sigma_c = x.T * gamma_diag * x
            covariance_matrices.append((sigma_c) / np.sum(gamma_values.T, axis = 0)[:,np.newaxis][i])
    # print("Covariance Matrices: \n",covariance_matrices)
    return mixing_coefficients,mean_vectors,covariance_matrices


### **INITIALISATION**

In [0]:
import numpy as np

#Initial mean vectors of the clusters
mean_vectors = []
random_indices = np.random.randint(0,len(X),3)
for index in random_indices:
  mean_vectors.append(X[index])

In [58]:
mean_vectors

[array([4.4, 3. , 1.3, 0.2]),
 array([6.8, 2.8, 4.8, 1.4]),
 array([7.3, 2.9, 6.3, 1.8])]

In [0]:
#Initial covariance matrices with variance = 1
covariance_matrices = []
for i in range(3):
  covariance_matrices.append(np.identity(4))

In [60]:
covariance_matrices

[array([[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]]), array([[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]]), array([[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]])]

In [0]:
mixing_coefficients = [1/3,1/3,1/3]

In [0]:
num_of_clusters = 3

### **EM ALGORITHM** 

In [63]:
for i in range(50):
  print("Iteration: ",i+1)
  #E step
  gamma_values = getGammaValues(X,num_of_clusters,mixing_coefficients,mean_vectors,covariance_matrices)
  # M step
  mixing_coefficients,mean_vectors,covariance_matrices = computeMixCoeffMeanVecAndCov(X,gamma_values,num_of_clusters)
  print("*"*100)

Iteration:  1
Mixing Coeffiients: 
 [0.35193383 0.45579879 0.19226738]
Mean Vectors: 
 [[5.01211302 3.37273871 1.56358273 0.28972894]
 [6.09652247 2.82729661 4.67125598 1.57338552]
 [6.7646089  3.02533955 5.60973735 1.97756215]]
****************************************************************************************************
Iteration:  2
Mixing Coeffiients: 
 [0.3342682  0.47511341 0.19061838]
Mean Vectors: 
 [[5.0061404  3.42522586 1.46660854 0.24829659]
 [6.04653782 2.81356012 4.59704926 1.54679106]
 [6.80494992 3.01979841 5.68486421 2.00103729]]
****************************************************************************************************
Iteration:  3
Mixing Coeffiients: 
 [0.33333329 0.47522227 0.19144444]
Mean Vectors: 
 [[5.00600006 3.42800013 1.46200002 0.24599999]
 [6.03551263 2.81699266 4.57311824 1.53643819]
 [6.82420892 3.00854452 5.73231119 2.02243379]]
****************************************************************************************************
Iteration:

In [66]:
mean_vectors

array([[5.006     , 3.428     , 1.462     , 0.246     ],
       [5.91496959, 2.77784365, 4.20155323, 1.29696685],
       [6.54454865, 2.94866115, 5.47955344, 1.98460496]])

### **PREDICTION**

In [0]:
cluster_labels = np.zeros((X.shape[0], num_of_clusters))
        
for i in range(num_of_clusters):
    cluster_labels [:,i] = mixing_coefficients[i] * mvn.pdf(X, mean_vectors[i], covariance_matrices[i])
cluster_labels  = cluster_labels.argmax(1)

In [65]:
from sklearn.metrics.cluster import adjusted_rand_score
adjusted_rand_score(y,cluster_labels)

0.9038742317748124

# **GMM SKLEARN**

In [0]:
from sklearn import  mixture
gmm = mixture.GaussianMixture(n_components=3, covariance_type='full')
gmm.fit(X)

GaussianMixture(covariance_type='full', init_params='kmeans', max_iter=100,
                means_init=None, n_components=3, n_init=1, precisions_init=None,
                random_state=None, reg_covar=1e-06, tol=0.001, verbose=0,
                verbose_interval=10, warm_start=False, weights_init=None)

In [0]:
gmm.means_

array([[5.006     , 3.428     , 1.462     , 0.246     ],
       [5.91697517, 2.77803998, 4.20523542, 1.29841561],
       [6.54632887, 2.94943079, 5.4834877 , 1.98716063]])

In [0]:
gmm.weights_

array([0.33333333, 0.30118609, 0.36548058])

In [0]:
y_pred = gmm.predict(X)

In [0]:
y_pred

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 2, 1, 2, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [0]:
from sklearn.metrics.cluster import adjusted_rand_score
adjusted_rand_score(y,y_pred)

0.9038742317748124

**By comparing the mean vectors and the adjusted rand score between our implementation and sklearn implementation, we conclude that results obtained are same.**